In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
inter_df = pd.read_csv('./prepped_data/bdinfo_baseline_maindata_inter_checkpoint_3.csv')
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')
print(inter_df.shape)
inter_df.head()

<string>:2: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.


(893973, 50)
Wall time: 2.88 s


,Unnamed: 0,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최근생성일자,사용승인_년도
0,0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20170209,2004.0
1,1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20190530,2008.0
2,2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20190530,2008.0
3,3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20171205,2005.0
4,4,11740-100179671,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20200428,2009.0


In [3]:
inter_df['층min'] = inter_df[['지번주소', '기준년월', '지상_층_수']].groupby(['지번주소', '기준년월']).transform('min')['지상_층_수']
inter_df['층max'] = inter_df[['지번주소', '기준년월', '지상_층_수']].groupby(['지번주소', '기준년월']).transform('max')['지상_층_수']
inter_df['층mean'] = inter_df[['지번주소', '기준년월', '지상_층_수']].groupby(['지번주소', '기준년월']).transform('mean')['지상_층_수']
inter_df['층median'] = inter_df[['지번주소', '기준년월', '지상_층_수']].groupby(['지번주소', '기준년월']).transform('median')['지상_층_수']

In [5]:
floor_df = inter_df[['지번주소', '기준년월', '층min', '층max', '층mean', '층median']]

In [6]:
print(floor_df.shape)
floor_df = floor_df.drop_duplicates(subset=['지번주소', '기준년월']).sort_values(['지번주소', '기준년월']).reset_index(drop=True)
print(floor_df.shape)
floor_df.head()

(893973, 6)
(732568, 6)


,지번주소,기준년월,층min,층max,층mean,층median
0,서울특별시 강남구 개포동 1163-4,201412,7,7,7.0,7.0
1,서울특별시 강남구 개포동 1163-4,201512,7,7,7.0,7.0
2,서울특별시 강남구 개포동 1163-4,201612,7,7,7.0,7.0
3,서울특별시 강남구 개포동 1163-4,201712,7,7,7.0,7.0
4,서울특별시 강남구 개포동 1163-4,201812,7,7,7.0,7.0


In [7]:
floor_df.to_csv('./추출한_건물정보/floor_min_max_mean_median.csv', index=False)

In [ ]:
def create_full_df_iteratively(df):
    # df is a pandas dataframe
    
    import pandas as pd
    import numpy as np
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        addr_df = df[df['지번주소'] == addr]
        
        min_record_year = addr_df['기준년도'].min()
        max_record_year = addr_df['기준년도'].max()
        unique_record_years = addr_df['기준년도'].unique()

        unique_approv_years = sorted(addr_df['사용승인년도'].unique().tolist())
        min_approved_year = addr_df['사용승인년도'].min()
        max_approved_year = addr_df['사용승인년도'].max()

        if min_approved_year <= 2006:
            min_year = 2006
        else:
            min_year = min_approved_year

        years_for_concat = [v for v in range(min_year, max_record_year) if v not in unique_record_years]
        if years_for_concat == []:
            dfs_list.append(addr_df)
            continue

        rows_list = []
        for year in years_for_concat:
            empty_row = addr_df.iloc[[0],:]
            empty_row['기준년도'] = year
            empty_row.iloc[0, 2:] = np.nan
            rows_list.append(empty_row)

        empty_df = pd.concat(rows_list)

        addr_concat_df = pd.concat([addr_df, empty_df]).sort_values(['기준년도']).reset_index(drop=True)

        addr_concat_df.fillna(method='ffill', inplace=True)
        addr_concat_df.fillna(method='bfill', inplace=True)
        
        dfs_list.append(addr_concat_df)
        count += 1
        if count % 5000 == 0:
            gc.collect()
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [ ]:
def parallelize(df, func, num_processors=2):
    # df is a pandas dataframe
    # func is the function to use
    # num_processors is the number of cpu cores
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [ ]:
%%time
completed_df = parallelize(floor_df, create_full_df_iteratively)
print(completed_df.shape)
completed_df.head()

In [ ]:
%%time
completed_df.to_csv('./prepped_data/bdinfo_floor_num.csv', index=False)